**Capital Asset Price Model (CAPM)** is one of the most important models in Finance and it describes the relationship between the expected return and risk of securities. we could use CAPM models to evluate the performance of stock. In the following projects, I would to analyse the stocks of following companies: Facebook, Netflix, Twitter and AT&T over the past 7 years.  

**Formula of CAPM:**

Expected Return of a specific secuirty = Risk Free Rate of Return + Beta between the stock and the Market (risk premium [incentive for investing in a risky security])

ri = rf + 	βi (rm - rf)

1. rf - Risk Free Rate of Return: a risk free asset could be a U.S. government 10 year Treasury Bill. 

2. rm - Market Portfolio Return: Market portfolio includes all securities in the market; A good representation of the market portfolio is the S&P 500 index. Viewed as a gauge of large-Cap US equities

3. βi - Beta represents the slope of the line regression line between Market returns vs Any individual stock returns; Beta is a measure of the volatility or systematic risk of a security or portfolio compared to the entire market. Beta is used in the CAPM and describes the relationship between systematic risk and expected return of assets.
  - βi = 0: Strongly correlated to market
  - βi < 1: defensive, the security is theoritically less volitile than the market
  - βi > 1: aggressive, the secuirty is theoritically more volatile than the market (for example Tesla's beta is around 1.26, indicate it is 26% more volatile than the market).

If an investor intended to invest in a specific stock, under the condition of specific set of stats we have (rf, βi, rm), we would know the expected returns by which could compensated the risk he/she is taking


In [44]:
# All the required Libs
import pandas as pd
import numpy as np
import pandas_datareader  as web
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import preprocessing # Normalization
import plotly.express as px # interactive graphing

In [17]:
# Prepare Data
pd.set_option("display.max_columns", None)
start_date = dt.datetime(2013,11,7)
end_date = dt.datetime(2020,11,7)
stock_df = web.DataReader(["FB","TWTR","NFLX","BA","T","MGM","TSLA","^GSPC"],data_source="yahoo", start = start_date, end = end_date)["Close"]
print(stock_df.head())

Symbols            FB       TWTR       NFLX          BA          T        MGM  \
Date                                                                            
2013-11-07  47.560001  44.900002  46.694286  131.509995  35.110001  18.580000   
2013-11-08  47.529999  41.650002  47.842857  133.490005  35.169998  19.090000   
2013-11-11  46.200001  42.900002  48.272858  132.529999  35.029999  19.200001   
2013-11-12  46.610001  41.900002  47.675713  132.330002  35.169998  19.260000   
2013-11-13  48.709999  42.599998  47.897144  133.169998  35.070000  19.700001   

Symbols          TSLA        ^GSPC  
Date                                
2013-11-07  27.954000  1747.150024  
2013-11-08  27.590000  1770.609985  
2013-11-11  28.940001  1771.890015  
2013-11-12  27.559999  1767.689941  
2013-11-13  27.740000  1782.000000  


In [ ]:
# Data Scaling: Normalization
def normalizer(df):
  x = df.values
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df = pd.DataFrame(x_scaled, columns = ["FB","TWTR","NFLX","BA","T","MGM","TSLA","^GSPC"])
  return df

normalized_data = normalizer(stock_df)
print(normalized_data)

In [47]:
# Interactive Stock Price Trend
def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns:
    fig.add_scatter(x = df.index, y=df[i], name = i)
  fig.show()

interactive_plot(stock_df, "Changes in Prices of different Securities")

In [77]:
interactive_plot(stock_df.pct_change().dropna(), "Volativity of the stocks")

In [63]:
# Get the betas

def beta_get(df):
  x = df.copy()
  # By set the period to 1, we get the daily returns
  x = x.pct_change(periods = 1, fill_method='ffill').dropna()
  beta_list = {}
  for col in df.columns[0:-1]:
    betas = np.polyfit(x["^GSPC"], x[col], 1)[0]
    beta_list[col] = betas
  return beta_list

print("The list of beta values", beta_get(stock_df))

The list of beta values {'FB': 1.1158861789228154, 'TWTR': 1.1831762053089148, 'NFLX': 1.0494230001203235, 'BA': 1.426391705477106, 'T': 0.7305377088952778, 'MGM': 1.6307254038413168, 'TSLA': 1.2880749914917848}


In [73]:
# Get the Daily Risk-Free returns: US 10-year Treasury Yeild
Non_risk_returns = web.DataReader(["^TNX"],data_source="yahoo", start = start_date, end = end_date)["Close"]
Non_risk_returns = Non_risk_returns/100
Newest_Non_risk_returns = Non_risk_returns.iloc[-1].values[0]
print(Newest_Non_risk_returns)

# Get the Daily Market returns:
Newest_market_returns = stock_df["^GSPC"].pct_change().iloc[-2]
print(Newest_market_returns)

0.008199999928474427
0.01946019413896294


In [74]:
# The CAPM Expected returns for each Individual Stocks
beta_FB = beta_get(stock_df)["FB"]
beta_TSLA = beta_get(stock_df)["TSLA"]
print(beta_FB)
def CAPM(beta, risk_free = Newest_Non_risk_returns, market = Newest_market_returns):
  expected_return = risk_free + beta*(market - risk_free)
  return expected_return

print("The expected return for FaceBook", CAPM(beta = beta_FB)*100,"%" )
print("The expected return for FaceBook", CAPM(beta = beta_TSLA)*100,"%" )

1.1158861789228154
The expected return for FaceBook 2.0765095019945257 %
The expected return for FaceBook 2.270397449034526 %


**Problems with This Model**
1. What if daily Market return is negative? which would probabily cause the expected return to be negative.

2. What if daily risk_free return > daily market return, which make the premium negative?

2. What if beta is negative?